In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.smpickle import load_pickle
import requests
import comp_utils
from comp_utils import day_ahead_market_times
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os
from datetime import datetime, timedelta
import pickle as pkl

In [2]:
with open("A-Team_key.txt") as f:
    api_key = f.read()

In [ ]:
with open('proxies.txt', 'r') as file:
        proxy = file.read()
proxies = {'http': proxy, 'https': proxy}

In [3]:
proxies = None

In [4]:
rebase_api_client = comp_utils.RebaseAPI(api_key = open("A-Team_key.txt").read(), proxy=proxies)

In [11]:
submission_data=pd.DataFrame({"datetime":comp_utils.day_ahead_market_times()})
submission_data['market_bid'] = np.random.randint(300, 1600, size=len(submission_data))
for quantile in range(10, 100, 10):
    submission_data[f'q{quantile}'] = np.random.randint(100, 1800, size=len(submission_data))
submission_data = comp_utils.prep_submission_in_json_format(submission_data)
submission_data

{'market_day': '2024-10-04',
 'submission': [{'timestamp': '2024-10-03T23:00:00+01:00',
   'market_bid': 1257,
   'probabilistic_forecast': {10: 181,
    20: 863,
    30: 1729,
    40: 1763,
    50: 737,
    60: 671,
    70: 1687,
    80: 484,
    90: 765}},
  {'timestamp': '2024-10-03T23:30:00+01:00',
   'market_bid': 480,
   'probabilistic_forecast': {10: 994,
    20: 1305,
    30: 1557,
    40: 900,
    50: 729,
    60: 311,
    70: 471,
    80: 842,
    90: 1249}},
  {'timestamp': '2024-10-04T00:00:00+01:00',
   'market_bid': 1008,
   'probabilistic_forecast': {10: 1353,
    20: 551,
    30: 1507,
    40: 1158,
    50: 765,
    60: 1322,
    70: 1314,
    80: 1173,
    90: 1054}},
  {'timestamp': '2024-10-04T00:30:00+01:00',
   'market_bid': 1327,
   'probabilistic_forecast': {10: 1459,
    20: 514,
    30: 896,
    40: 1519,
    50: 575,
    60: 1589,
    70: 1723,
    80: 589,
    90: 1141}},
  {'timestamp': '2024-10-04T01:00:00+01:00',
   'market_bid': 584,
   'probabilistic_for

In [13]:
import numpy as np

def convert_int32_to_int(data):
    if isinstance(data, dict):
        return {k: convert_int32_to_int(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [convert_int32_to_int(item) for item in data]
    elif isinstance(data, np.int32):
        return int(data)
    else:
        return data

# Convert int32 values in submission_data
submission_data = convert_int32_to_int(submission_data)

In [14]:
# Now submit the data
rebase_api_client.submit(submission_data)

<Response [200]>
{"id":"7f3a5b87-922c-49f0-b2f6-932bed801fd7","participant_id":"d742556b-2b48-493e-8959-7e6af7572f42","prediction_date":"2024-10-04","solution":{"market_day":"2024-10-04","submission":[{"market_bid":1257.0,"timestamp":"2024-10-03T22:00:00+00:00","probabilistic_forecast":{"10":181,"20":863,"30":1729,"40":1763,"50":737,"60":671,"70":1687,"80":484,"90":765}},{"market_bid":480.0,"timestamp":"2024-10-03T22:30:00+00:00","probabilistic_forecast":{"10":994,"20":1305,"30":1557,"40":900,"50":729,"60":311,"70":471,"80":842,"90":1249}},{"market_bid":1008.0,"timestamp":"2024-10-03T23:00:00+00:00","probabilistic_forecast":{"10":1353,"20":551,"30":1507,"40":1158,"50":765,"60":1322,"70":1314,"80":1173,"90":1054}},{"market_bid":1327.0,"timestamp":"2024-10-03T23:30:00+00:00","probabilistic_forecast":{"10":1459,"20":514,"30":896,"40":1519,"50":575,"60":1589,"70":1723,"80":589,"90":1141}},{"market_bid":584.0,"timestamp":"2024-10-04T00:00:00+00:00","probabilistic_forecast":{"10":1305,"20":6

FileNotFoundError: [Errno 2] No such file or directory: 'logs/sub_20241003-094830.txt'

In [83]:
def Update_Weather_Data(models):
    csv_path = f'{models}.csv'
    # Get the weather data
    basic_variables = "Temperature, WindSpeed, WindSpeed:100, WindDirection:100, CloudCover, RelativeHumidity, PressureReducedMSL, SolarDownwardRadiation, TotalPrecipitation"
    url = "https://api.rebase.energy/weather/v2/query"
    lats = [52.4872562, 52.8776682, 52.1354277, 52.4880497, 51.9563696, 52.2499177, 52.6416477, 52.2700912, 52.1960768, 52.7082618, 52.4043468, 52.0679429, 52.024023, 52.7681276, 51.8750506, 52.5582373, 52.4478922, 52.5214863, 52.8776682, 52.0780721]
    lons = [0.4012455, 0.7906532, -0.2640343, -0.1267052, 0.6588173, 1.3894081, 1.3509559, 0.7082557, 0.1534462, 0.7302284, 1.0762977, 1.1751747, 0.2962684, 0.1699257, 0.9115028, 0.7137489, 0.1204872, 1.5706825, 1.1916542, -0.0113488]
    body= {'model': models, 'latitude': lats,
        'longitude': lons,
          'variables': basic_variables, 'output-format': 'json', 'forecast-horizon': "latest"}
    try:
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            print(f"Loaded existing CSV with {len(df)} rows")
        else:
            df = pd.DataFrame(columns=["ref_datetime","valid_datetime",basic_variables,"latitude","longitude"])
            print("Creating new CSV file")
        response = requests.post(url, json=body, headers={'Authorization': f'{api_key}'})
        for point in range(len(response.json())):
            new_df = pd.DataFrame(response.json()[point])
            new_df['latitude'] = lats[point]
            new_df['longitude'] = lons[point]
            df = pd.concat([df, new_df])
        df.drop_duplicates(inplace=True)
        df.to_csv(csv_path, index=False)

    except requests.exceptions.RequestException as e:
        print(e)
        return None
    return None


In [4]:
rebase_api_client.get_pes10_nwp(model="DWD_ICON-EU")

{'model': 'DWD_ICON-EU', 'latitude': [52.4872562, 52.8776682, 52.1354277, 52.4880497, 51.9563696, 52.2499177, 52.6416477, 52.2700912, 52.1960768, 52.7082618, 52.4043468, 52.0679429, 52.024023, 52.7681276, 51.8750506, 52.5582373, 52.4478922, 52.5214863, 52.8776682, 52.0780721], 'longitude': [0.4012455, 0.7906532, -0.2640343, -0.1267052, 0.6588173, 1.3894081, 1.3509559, 0.7082557, 0.1534462, 0.7302284, 1.0762977, 1.1751747, 0.2962684, 0.1699257, 0.9115028, 0.7137489, 0.1204872, 1.5706825, 1.1916542, -0.0113488], 'variables': 'SolarDownwardRadiation, CloudCover, Temperature', 'type': 'points', 'output-format': 'json', 'forecast-horizon': 'latest'}
https://api.rebase.energy/weather/v2/query
IjDBb9M2CjNgPKyyV3mwvFuz2g5V0CkcOQy0-0DbryQ
200


,valid_datetime,ref_datetime,SolarDownwardRadiation,CloudCover,Temperature,point,latitude,longitude
0,2024-10-02T12:00:00Z,2024-10-02T12:00:00Z,119.61,1.00,13.99,0,52.487256,0.401245
1,2024-10-02T13:00:00Z,2024-10-02T12:00:00Z,219.60,0.92,14.12,0,52.487256,0.401245
2,2024-10-02T14:00:00Z,2024-10-02T12:00:00Z,86.32,0.86,13.96,0,52.487256,0.401245
3,2024-10-02T15:00:00Z,2024-10-02T12:00:00Z,90.85,0.89,13.40,0,52.487256,0.401245
4,2024-10-02T16:00:00Z,2024-10-02T12:00:00Z,63.36,0.72,13.45,0,52.487256,0.401245
...,...,...,...,...,...,...,...,...
88,2024-10-07T00:00:00Z,2024-10-02T12:00:00Z,1.88,0.43,11.57,19,52.078072,-0.011349
89,2024-10-07T03:00:00Z,2024-10-02T12:00:00Z,0.49,0.34,11.19,19,52.078072,-0.011349
90,2024-10-07T06:00:00Z,2024-10-02T12:00:00Z,48.66,1.00,11.52,19,52.078072,-0.011349
91,2024-10-07T09:00:00Z,2024-10-02T12:00:00Z,73.04,1.00,13.45,19,52.078072,-0.011349


In [84]:
List_of_models = ['DWD_ICON-EU']
for model in List_of_models:
    Update_Weather_Data(model)
    print(f"Updated {model} weather data")

Creating new CSV file
Updated DWD_ICON-EU weather data


In [7]:
url = "https://api.rebase.energy/weather/v2/query"
body= {'model': 'DWD_ICON-EU', 'latitude': [52.4872562, 52.8776682],
        'longitude': [0.4012455, 0.7906532],
          'variables': 'SolarDownwardRadiation, Temperature', 'output-format': 'json', 'forecast-horizon': "full",}
          # 'start-date': '2024-09-27', 'end-date': '2024-09-29','reference-time-freq': '6H'}

response = requests.post(url, json=body, headers={'Authorization': api_key})
lats = [52.4872562, 52.8776682, 52.1354277, 52.4880497, 51.9563696, 52.2499177, 52.6416477, 52.2700912, 52.1960768, 52.7082618, 52.4043468, 52.0679429, 52.024023, 52.7681276, 51.8750506, 52.5582373, 52.4478922, 52.5214863, 52.8776682, 52.0780721]
lons = [0.4012455, 0.7906532, -0.2640343, -0.1267052, 0.6588173, 1.3894081, 1.3509559, 0.7082557, 0.1534462, 0.7302284, 1.0762977, 1.1751747, 0.2962684, 0.1699257, 0.9115028, 0.7137489, 0.1204872, 1.5706825, 1.1916542, -0.0113488]
df = pd.DataFrame()
print(response.json())
for point in range(len(response.json())):
    new_df = pd.DataFrame(response.json()[point])
    new_df['latitude'] = lats[point]
    new_df['longitude'] = lons[point]
    df = pd.concat([df, new_df])
   

[{'ref_datetime': ['2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z', '2024-10-02T12:00:00Z

3Tage und 6h

full und latest keinen unterschied

In [63]:
df

,ref_datetime,valid_datetime,SolarDownwardRadiation,CloudCover,Temperature,latitude,longitude
0,2024-10-02T12:00:00Z,2024-10-02T12:00:00Z,119.61,1.00,13.99,52.487256,0.401245
1,2024-10-02T12:00:00Z,2024-10-02T13:00:00Z,219.60,0.92,14.12,52.487256,0.401245
2,2024-10-02T12:00:00Z,2024-10-02T14:00:00Z,86.32,0.86,13.96,52.487256,0.401245
3,2024-10-02T12:00:00Z,2024-10-02T15:00:00Z,90.85,0.89,13.40,52.487256,0.401245
4,2024-10-02T12:00:00Z,2024-10-02T16:00:00Z,63.36,0.72,13.45,52.487256,0.401245
...,...,...,...,...,...,...,...
88,2024-10-02T12:00:00Z,2024-10-07T00:00:00Z,1.88,0.43,11.57,52.078072,-0.011349
89,2024-10-02T12:00:00Z,2024-10-07T03:00:00Z,0.49,0.34,11.19,52.078072,-0.011349
90,2024-10-02T12:00:00Z,2024-10-07T06:00:00Z,48.66,1.00,11.52,52.078072,-0.011349
91,2024-10-02T12:00:00Z,2024-10-07T09:00:00Z,73.04,1.00,13.45,52.078072,-0.011349


In [16]:
df[0:1]["ref_datetime"]

0    [2024-10-02T12:00:00Z, 2024-10-02T12:00:00Z, 2...
Name: ref_datetime, dtype: object

In [ ]:
df = pd.DataFrame()
    for point in range(len(data)):
      new_df = pd.DataFrame(data[point])
      new_df["point"] = point
      new_df["latitude"] = lats[point]
      new_df["longitude"] = lons[point]
      df = pd.concat([df,new_df])

In [ ]:
# everything for 
rebase_api_client.get_hornsea_dwd()

In [69]:
def update_day_ahead_price_csv(min_date, csv_path='day_ahead_price.csv'):
    """
    Fetches day-ahead prices from the Rebase API and appends them to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        min_date = datetime.strptime(min_date, '%Y-%m-%d').date()
        today = datetime.today().date()
        
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                min_date = df['settlement_date'].max()
                # df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'settlement_date', 'settlement_period', 'price'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'settlement_date', 'settlement_period', 'price'])
            print("Creating new CSV file")
        
        # Generate dates to updat
        print(f"min_date: {min_date}, today: {today}")
        dates_to_update = pd.date_range(start=min_date, end=today, freq='D')
        try:
            for date in dates_to_update:
                date_str = date.strftime('%Y-%m-%d')
                prices = rebase_api_client.get_day_ahead_price(date_str)
                if len(prices):
                    print(f"Fetched {len(prices)} prices for {date_str}")
                    new_df = pd.DataFrame(prices)
                    # new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                    df = pd.concat([df, new_df], ignore_index=True)
                else:
                    print(f"No data returned for {date_str}")
            df.drop_duplicates(inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [80]:
def update_market_index_csv(min_date, csv_path='market_index.csv'):
    """
    Fetches market index data from the Rebase API and appends it to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        min_date = datetime.strptime(min_date, '%Y-%m-%d').date()
        today = datetime.today().date()
        
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                min_date = df['timestamp_utc'].max()
                df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'index'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'index'])
            print("Creating new CSV file")
        
        # Generate dates to update
        print(f"min_date: {min_date}, today: {today}")
        dates_to_update = pd.date_range(start=min_date, end=today, freq='D')
        try:
            for date in dates_to_update:
                date_str = date.strftime('%Y-%m-%d')
                indices = rebase_api_client.get_market_index(date_str)
                if len(indices):
                    print(f"Fetched {len(indices)} indices for {date_str}")
                    new_df = pd.DataFrame(indices)
                    new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                    df = pd.concat([df, new_df], ignore_index=True)
                else:
                    print(f"No data returned for {date_str}")
            df.drop_duplicates(subset=['timestamp_utc'], inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [82]:
def update_imbalance_price_csv(min_date, csv_path='imbalance_price.csv'):
    """
    Fetches imbalance prices from the Rebase API and appends them to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        min_date = datetime.strptime(min_date, '%Y-%m-%d').date()
        today = datetime.today().date()
        
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                min_date = df['timestamp_utc'].max()
                df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'price'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'price'])
            print("Creating new CSV file")
        
        # Generate dates to update
        print(f"min_date: {min_date}, today: {today}")
        dates_to_update = pd.date_range(start=min_date, end=today, freq='D')
        try:
            for date in dates_to_update:
                date_str = date.strftime('%Y-%m-%d')
                prices = rebase_api_client.get_imbalance_price(date_str)
                if len(prices):
                    print(f"Fetched {len(prices)} prices for {date_str}")
                    new_df = pd.DataFrame(prices)
                    new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                    df = pd.concat([df, new_df], ignore_index=True)
                else:
                    print(f"No data returned for {date_str}")
            df.drop_duplicates(subset=['timestamp_utc'], inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [84]:
def update_wind_total_prduction_csv(min_date, csv_path='wind_total_production.csv'):
    """
    Fetches total wind production data from the Rebase API and appends it to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        min_date = datetime.strptime(min_date, '%Y-%m-%d').date()
        today = datetime.today().date()
        
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                min_date = df['timestamp_utc'].max()
                df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'production'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'production'])
            print("Creating new CSV file")
        
        # Generate dates to update
        print(f"min_date: {min_date}, today: {today}")
        dates_to_update = pd.date_range(start=min_date, end=today, freq='D')
        try:
            for date in dates_to_update:
                date_str = date.strftime('%Y-%m-%d')
                productions = rebase_api_client.get_wind_total_production(date_str)
                if len(productions):
                    print(f"Fetched {len(productions)} productions for {date_str}")
                    new_df = pd.DataFrame(productions)
                    new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                    df = pd.concat([df, new_df], ignore_index=True)
                else:
                    print(f"No data returned for {date_str}")
            df.drop_duplicates(subset=['timestamp_utc'], inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [86]:
def update_solar_total_prodcution_csv(min_date, csv_path='solar_total_production.csv'):
    """
    Fetches total solar production data from the Rebase API and appends it to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        min_date = datetime.strptime(min_date, '%Y-%m-%d').date()
        today = datetime.today().date()
        
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                min_date = df['timestamp_utc'].max()
                df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'production'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'production'])
            print("Creating new CSV file")
        
        # Generate dates to update
        print(f"min_date: {min_date}, today: {today}")
        dates_to_update = pd.date_range(start=min_date, end=today, freq='D')
        try:
            for date in dates_to_update:
                date_str = date.strftime('%Y-%m-%d')
                productions = rebase_api_client.get_solar_total_production(date_str)
                if len(productions):
                    print(f"Fetched {len(productions)} productions for {date_str}")
                    new_df = pd.DataFrame(productions)
                    new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                    df = pd.concat([df, new_df], ignore_index=True)
                else:
                    print(f"No data returned for {date_str}")
            df.drop_duplicates(subset=['timestamp_utc'], inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [94]:
def update_solar_wind_forecast_csv(min_date, csv_path='solar_wind_forecast.csv'):
    """
    Fetches solar and wind forecast data from the Rebase API and appends it to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        min_date = datetime.strptime(min_date, '%Y-%m-%d').date()
        today = datetime.today().date()
        
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                min_date = df['timestamp_utc'].max()
                df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'solar_forecast', 'wind_forecast'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'solar_forecast', 'wind_forecast'])
            print("Creating new CSV file")
        
        # Generate dates to update
        print(f"min_date: {min_date}, today: {today}")
        dates_to_update = pd.date_range(start=min_date, end=today, freq='D')
        try:
            for date in dates_to_update:
                date_str = date.strftime('%Y-%m-%d')
                forecasts = rebase_api_client.get_solar_wind_forecast(date_str)
                if len(forecasts):
                    print(f"Fetched {len(forecasts)} forecasts for {date_str}")
                    new_df = pd.DataFrame(forecasts)
                    new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                    df = pd.concat([df, new_df], ignore_index=True)
                else:
                    print(f"No data returned for {date_str}")
            df.drop_duplicates(subset=['timestamp_utc'], inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [18]:
def update_margin_forecast(csv_path='margin_forecast.csv'):
    """
    Fetches solar and wind forecast data from the Rebase API and appends it to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['forecast_date', 'publish_time_utc', 'margin'])
        else:
            df = pd.DataFrame(columns=['forecast_date', 'publish_time_utc', 'margin'])
            print("Creating new CSV file")
        
        # Generate dates to update
        try:
            forecasts = rebase_api_client.get_margin_forecast()
            if len(forecasts):
                print(f"Fetched {len(forecasts)} forecasts")
                new_df = pd.DataFrame(forecasts)
                # new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                df = pd.concat([df, new_df], ignore_index=True)
            else:
                print(f"No data returned for")
            df.drop_duplicates(inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for : {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [4]:
def update_day_ahead_demand_forecast(csv_path='day_ahead_demand_forecast.csv'):
    """
    Fetches solar and wind forecast data from the Rebase API and appends it to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'settlement_date', 'settlement_period', 'boundary',
                    'publish_time_utc', 'transmission_system_demand', 'national_demand'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'settlement_date', 'settlement_period', 'boundary',
                'publish_time_utc', 'transmission_system_demand', 'national_demand'])
            print("Creating new CSV file")
        
        # Generate dates to update
        try:
            forecasts = rebase_api_client.get_day_ahead_demand_forecast()
            if len(forecasts):
                print(f"Fetched {len(forecasts)} forecasts")
                new_df = pd.DataFrame(forecasts)
                df = pd.concat([df, new_df], ignore_index=True)
            else:
                print(f"No data returned for")
            df.drop_duplicates(inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for : {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [112]:
rebase_api_client.get_solar_wind_forecast('2024-09-01').columns

c:\Temp\EnergyProject\HEFTcom24\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srv01prox.corp.trumpf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Index(['timestamp_utc', 'settlement_date', 'settlement_period', 'solar_mw',
       'wind_offshore_mw', 'wind_onshore_mw'],
      dtype='object')

In [20]:
df1 = pd.DataFrame(rebase_api_client.get_day_ahead_demand_forecast())

c:\Temp\EnergyProject\HEFTcom24\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srv01prox.corp.trumpf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>


In [21]:
df1.columns

Index(['timestamp_utc', 'settlement_date', 'settlement_period', 'boundary',
       'publish_time_utc', 'transmission_system_demand', 'national_demand'],
      dtype='object')

In [10]:
times = day_ahead_market_times()

In [14]:
times

DatetimeIndex(['2024-10-02 23:00:00+01:00', '2024-10-02 23:30:00+01:00',
               '2024-10-03 00:00:00+01:00', '2024-10-03 00:30:00+01:00',
               '2024-10-03 01:00:00+01:00', '2024-10-03 01:30:00+01:00',
               '2024-10-03 02:00:00+01:00', '2024-10-03 02:30:00+01:00',
               '2024-10-03 03:00:00+01:00', '2024-10-03 03:30:00+01:00',
               '2024-10-03 04:00:00+01:00', '2024-10-03 04:30:00+01:00',
               '2024-10-03 05:00:00+01:00', '2024-10-03 05:30:00+01:00',
               '2024-10-03 06:00:00+01:00', '2024-10-03 06:30:00+01:00',
               '2024-10-03 07:00:00+01:00', '2024-10-03 07:30:00+01:00',
               '2024-10-03 08:00:00+01:00', '2024-10-03 08:30:00+01:00',
               '2024-10-03 09:00:00+01:00', '2024-10-03 09:30:00+01:00',
               '2024-10-03 10:00:00+01:00', '2024-10-03 10:30:00+01:00',
               '2024-10-03 11:00:00+01:00', '2024-10-03 11:30:00+01:00',
               '2024-10-03 12:00:00+01:00', '2024-1

In [ ]:
rebase_api_client.get_hornsea_dwd()

In [6]:
rebase_api_client.get_day_ahead_price(day="2024-09-30")

ConnectTimeout: HTTPSConnectionPool(host='api.rebase.energy', port=443): Max retries exceeded with url: /challenges/data/day_ahead_price?day=2024-09-30 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001961D7E5010>, 'Connection to api.rebase.energy timed out. (connect timeout=None)'))

In [ ]:
rebase_api_client.get_day_ahead_price(day="2024-09-30")

In [ ]:
latest_dwd_Hornsea1 = comp_utils.weather_df_to_xr(rebase_api_client.get_hornsea_dwd())
latest_dwd_solar = comp_utils.weather_df_to_xr(rebase_api_client.get_pes10_nwp("DWD_ICON-EU"))

In [ ]:
test_date = "2023-05-05"

rebase_api_client.get_variable(day=test_date,variable="day_ahead_price").head()

In [5]:
rebase_api_client.get_variable(day=test_date,variable="imbalance_price").head()
# rebase_api_client.get_variable(day=test_date,variable="wind_total_production").head()
# rebase_api_client.get_variable(day=test_date,variable="solar_total_production").head()

ValueError: DataFrame constructor not properly called!

In [4]:
import requests
API_key = open("team_key.txt").read()
url = "https://api.rebase.energy/weather/v2/query"
headers = {"Authorization": API_key}
params = {
    'model': 'DWD_ICON-EU',
    'start-date': '2024-09-30',
    'end-date': '2024-10-03',
    'reference-time-freq': '6H',
    'latitude': '60.1, 61.2, 59.33', 
    'longitude': '17.2, 13.1, 18.05',
    'variables': 'Temperature, WindSpeed',
    'output-format': 'json'   
}
response = requests.get(url, headers=headers, params=params)


In [5]:
response.json()

{'message': 'Unauthorized'}

In [29]:
import requests

your_api_key = '' # insert your API key here
url = "https://api.rebase.energy/weather/v2/query"
headers = {"Authorization": API_key,
           'Content-Type': 'application/json'}
params = {
    'model': 'DWD_ICON-EU',
    'start-date': '2024-09-30',
    'end-date': '2024-10-03',
    'reference-time-freq': '6H',
    'latitude': '60.1, 61.2, 59.33', 
    'longitude': '17.2, 13.1, 18.05',
    'variables': 'Temperature, WindSpeed',
    'output-format': 'json'
}
response = requests.post(url,
                         headers=headers,
                         json=params,)
                        #  content_type='application/json')

In [31]:
response.json()

{'message': 'Unauthorized'}

In [32]:
if response.status_code == 200:
    print("Request successful.")
    print(response.json())  # Print the JSON response
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")

Request failed with status code 401: {"message":"Unauthorized"}



In [33]:
response.json()

{'message': 'Unauthorized'}

In [39]:
import requests

your_api_key = '' # insert your API key here
url = "https://api.rebase.energy/weather/v2/point/historical"
headers = {"Authorization": your_api_key}
params = {
    'model': 'DWD_ICON-EU',
    'start-date': '2024-09-01',
    'end-date': '2024-09-09',
    'reference-time-freq': '6H',
    'forecast-horizon': 'full',
    'latitude': '60.1, 61.2, 59.33', 
    'longitude': '17.2, 13.1, 18.05',
    'variables': 'Temperature, WindSpeed:10'
}
response = requests.get(url, headers=headers, params=params)

In [40]:
response.json()

{'message': 'Unauthorized'}

In [51]:
import requests
API_key = open("team_key.txt").read()
url = "https://api.rebase.energy/challenges/data/day_ahead_price"
headers = {"Authorization": f"Bearer {API_key}"}  # Prefix the API key with 'Bearer '
params = {
    "date": "2024-10-01"
}
response = requests.get(url, headers=headers, params=params)

In [52]:
response.json()

'Bearer token is invalid'

In [53]:
import requests
API_key = open("team_key.txt").read()
url = "https://api.rebase.energy/challenges/data/market_index"
headers = {"Authorization": f"Bearer {API_key}"}  # Prefix the API key with 'Bearer '
params = {
    "date": "2024-10-01"
}
response = requests.get(url, headers=headers, params=params)

In [54]:
response.json()

'Bearer token is invalid'

In [55]:
import requests
API_key = open("team_key.txt").read()
url = "https://api.rebase.energy/challenges/data/imbalance_price"
headers = {"Authorization": f"Bearer {API_key}"}  # Prefix the API key with 'Bearer '
params = {
    "date": "2024-10-01"
}
response = requests.get(url, headers=headers, params=params)

In [56]:
response.json()

'Bearer token is invalid'

In [ ]:
import requests
API_key = open("team_key.txt").read()
url = "https://api.rebase.energy/challenges/data/imbalance_price"
headers = {"Authorization": f"Bearer {API_key}"}  # Prefix the API key with 'Bearer '
params = {
    "date": "2024-10-01"
}
response = requests.get(url, headers=headers, params=params)